### Задание

взять данные из
https://www.kaggle.com/datasets/mrapplexz/bashim-quotes
обучить модель GPT для генерации своих цитат

взять новостные данные из
https://github.com/natasha/corus
load_lenta2
нам понадобиться сам текст и заголовок
обучить модель T5/ или GPT для генерации заголовков для статей

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, T5ForConditionalGeneration

In [4]:
DATASET_PATH = 'dataset.jsonl/dataset.jsonl'

#открываем файл
with open(DATASET_PATH) as f: 
   # считываем .jsonl file как Pandas DataFrame
    df = pd.read_json(DATASET_PATH, lines=True).set_index('id')

In [5]:
df.head()

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...


In [6]:
df.shape

(81497, 3)

Препроцессинг

In [7]:
#ограничиваем объем данных столбца text
data = df.loc[:5000, 'text']

In [8]:
train, test = train_test_split(data, test_size=0.15)

In [11]:
#Делаем препроцессинг для всего text из каждого набора данных и собираем в отдельные файлы
import re

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w', encoding="utf8")
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\[\w+\]", "", summary)
        summary = re.sub(r"<[\w+,\!, -]>", "", summary)
        summary = re.sub(r"<\w+>", "", summary)
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [12]:
build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [14]:
with open('train_dataset.txt', encoding="utf8") as f: 
    train_dataset = f.read()
train_dataset

'* gvy is away: I"m bussy  автобусный :)  <ann-> Какой-то он живой для будущего покойника  ann-: будущие покойники все живые <ann-> anondll ?  ann-: ну вот так. а ты не догадывалась?   Бтв, НетЛенко. Я Вам уже говорил, что у меня на подходе есть Friends of the Random Summer Compilation в lossless?  нед.  Так вот.  У меня на подходе есть Friends of the Random Summer Compilation в lossless.   все у меня в жизни через жопу.....  ужос!  что и секс???  <fox-o> "все Ваши деньги будут использованы в некоммерческих целях нашего личного обогащения"   народ, таблицы маршрутизации всем админам снятся, или пора бы к психиатору? :(   expertsexchange.com - не сайт специалистов по смене пола, а сообщество по обмену знаниями между программистами  magius wrote: - Что такое "поддержка пользователей"? - А это если у пользователя вся инфа накрылась медным тазом, то ты подходишь, ложишь руку на плечо и говоришь: "Как я вас понимаю!"  Semikolenov wrote: смеешься, да? Я сейчас по кусочкам мужику восстанавлив

In [15]:
print("Train length: "+ str(len(train)))
print("Test length: "+ str(len(test)))

Train length: 1197
Test length: 212


Загружаем токенайзер и создаем датасеты

In [16]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [21]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

C:\Users\Uliana\anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Загружаем и обучаем модель

In [22]:
model = AutoModelForCausalLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [23]:
#аргументы для обучения
training_args = TrainingArguments(
    output_dir="gdrive/MyDrive/GPT/gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [25]:
trainer.train()

C:\Users\Uliana\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 465
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 351


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=351, training_loss=4.362702198517629, metrics={'train_runtime': 2517.9089, 'train_samples_per_second': 0.554, 'train_steps_per_second': 0.139, 'total_flos': 91125596160000.0, 'train_loss': 4.362702198517629, 'epoch': 3.0})

Сохраненяем модель и токенайзер

In [26]:
trainer.save_model()

Saving model checkpoint to gdrive/MyDrive/GPT/gpt2-chief
Configuration saved in gdrive/MyDrive/GPT/gpt2-chief\config.json
Model weights saved in gdrive/MyDrive/GPT/gpt2-chief\pytorch_model.bin


In [27]:
tokenizer.save_pretrained('gpt2-chief')

tokenizer config file saved in gdrive/MyDrive/GPT/gpt2-chief\tokenizer_config.json
Special tokens file saved in gdrive/MyDrive/GPT/gpt2-chief\special_tokens_map.json


('gdrive/MyDrive/GPT/gpt2-chief\\tokenizer_config.json',
 'gdrive/MyDrive/GPT/gpt2-chief\\special_tokens_map.json',
 'gdrive/MyDrive/GPT/gpt2-chief\\vocab.json',
 'gdrive/MyDrive/GPT/gpt2-chief\\merges.txt',
 'gdrive/MyDrive/GPT/gpt2-chief\\added_tokens.json',
 'gdrive/MyDrive/GPT/gpt2-chief\\tokenizer.json')

In [28]:
model.save_pretrained('model_gpt_chf')

Configuration saved in gdrive/MyDrive/GPT/model_gpt_chf\config.json
Model weights saved in gdrive/MyDrive/GPT/model_gpt_chf\pytorch_model.bin


Генерируем текст

In [29]:
#загружаем токенайзер и модель
tokenizer = AutoTokenizer.from_pretrained("gpt2-chief")
model1 = AutoModelForCausalLM.from_pretrained("model_gpt_chf")

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file gdrive/MyDrive/GPT/model_gpt_chf\config.json
Model config GPT2Config {
  "_name_or_path": "gdrive/MyDrive/GPT/model_gpt_chf",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "s

In [30]:
prefix = "Место силы"
tokens = tokenizer(prefix, return_tensors='pt')
size = tokens['input_ids'].shape[1]
output = model.generate( 
    **tokens, 
    #end_token=end_token_id,
    do_sample=False,
    max_length=+50,
#     max_length=size1+50, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Место силы" - это место, где можно выпить кофе и закусить бутербродами с колбасой.  <@Koi-ZaRa> А что такое "место силы"? * Koi-ZaRa


Оригинальное определение места силы :) На самом деле так называются соревнования по силовому спорту. Но колбаса с кофе тоже пойдет!

взять новостные данные из https://github.com/natasha/corus load_lenta2 нам понадобиться сам текст и заголовок обучить модель T5/ или GPT для генерации заголовков для статей

In [31]:
!pip install corus
!pip install datasets

In [48]:
import pandas as pd
from datasets import load_from_disk

In [34]:
from corus import load_buriy_news

paths = [
    'news-articles-2014.tar.bz2',
    'news-articles-2015-part1.tar.bz2',
    'news-articles-2015-part2.tar.bz2'
]
records = (
    record
    for path in paths
    for record in load_buriy_news(path)
)
next(records)

BuriyRecord(
    timestamp=datetime.datetime(2014, 8, 22, 15, 15),
    url='http://www.ntv.ru/novosti/1200239/',
    edition=None,
    topics='novosti',
    title='Россияне на юношеских Олимпийских играх в пятницу завоевали восемь медалей',
    text='В пятницу российские спортсмены на\nюношеских Олимпийских играх в китайском Нанкине\nзавоевали еще восемь наград \x97 две золотые, три серебряные и три бронзовые.\nТяжелоатлет Хетаг Хугаев завоевал золото в весовой категории до 85 кг. Розалия Насретдинова первенствовала в плавании на дистанции 50 метров вольным стилем.\nЕвгений Рылов принес россиянам серебро в плавании на дистанции 200 метров на спине. Он же вместе с Александром Садовниковым, Розалией Насретдиновой и Дарьей Устиновой стал вторым в смешанной комбинированной эстафете 4х100 м.\nШтангистка Светлана Щербакова завоевала серебро в весовой категории свыше 63 кг.\nКроме того, та же Дарья Устинова завоевала бронзу в плавании на дистанции 50 м вольным стилем, а Антон Чупков \x97 нагр

In [35]:
#собираем название и текст новостей и преобразуем в датафрейм
data = [(record.title, record.text) for record in records]

In [36]:
df_news = pd.DataFrame({'title': [record[0] for record in data], 'text': [record[1] for record in data]})
df_news.head()

,title,text
0,Российские журналисты попали под минометный об...,"Пётр Михайлов\n, сотрудник пресс-службы провоз..."
1,ЕС призывает РФ вывести гуманитарный конвой с ...,"Себастьен Брабант\n, представитель ЕС: «Мы сож..."
2,Яценюк: у нас есть своя гуманитарная помощь,"Арсений Яценюк заявил, что Украина не нуждаетс..."
3,Музей варежки открывает выставку художника-инв...,Сегодня в петербургском музее варежки открылас...
4,Тихонов станет единственным кандидатом от Росс...,Александр Тихонов (на фото) будет единственным...


In [37]:
df_news.shape

(2154800, 2)

In [38]:
#смотрим максимальную длину текста и названия статьи
max(len(i) for i in df_news['text'].values), max(len(i) for i in df_news['title'].values)

(717050, 252)

In [39]:
#ограничиваем объем данных
data_news = df_news[:500]

In [40]:
df_train, df_test = train_test_split(data_news, test_size=0.15)

In [41]:
#складываем объекты в Dataset (для huggingface)
from datasets import Dataset
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)
dataset_train, dataset_test

(Dataset({
     features: ['title', 'text'],
     num_rows: 425
 }),
 Dataset({
     features: ['title', 'text'],
     num_rows: 75
 }))

In [42]:
#создаем токенайзер
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)

#ограничиваем максимальную длину текста и новости
max_len_txt = 400
max_len_tlt = 50

#токенизируем вход
def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_len_txt)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tlt)
    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

#токенизируем данные из датасетов
dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

#столбцы ['input_ids', 'attention_mask', 'labels'] переводим в формат numpy
dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/828k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

loading file spiece.model from cache at C:\Users\Uliana/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\spiece.model
loading file tokenizer.json from cache at C:\Users\Uliana/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\Uliana/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\Uliana/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\tokenizer_config.json


  0%|          | 0/54 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [43]:
dataset_train, dataset_test

(Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 425
 }),
 Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 75
 }))

In [44]:
#сохранение подготовленных датасетов
dataset_train.save_to_disk('gazeta-train')
dataset_test.save_to_disk('gazeta-test')

Загрузка и обучение модели

In [45]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"
model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

loading configuration file config.json from cache at C:\Users\Uliana/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\config.json
Model config T5Config {
  "_name_or_path": "cointegrated/rut5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 2,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 200,
  "model_type": "t5",
  "num_beams": 5,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at C:\Users\Uliana/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\pytorch_model.bin
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at IlyaGusev/rut5_base_sum_gazeta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.


In [46]:
#аргументы для обучения
output_dir = 'gazeta/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_steps=1000, # How often to save a checkpoint
    remove_unused_columns=True, # Removes useless columns from the dataset
    eval_steps=500, # How often to run evaluation on the val_set
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [49]:
dataset_train = load_from_disk("gazeta-train")
dataset_test = load_from_disk("gazeta-test")
dataset_train, dataset_test

(Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 425
 }),
 Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 75
 }))

In [50]:
#обучаем модель
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title. If text, title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
C:\Users\Uliana\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 425
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 639


Step,Training Loss
500,4.472800




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=639, training_loss=3.695304787028377, metrics={'train_runtime': 7662.0817, 'train_samples_per_second': 0.166, 'train_steps_per_second': 0.083, 'total_flos': 677081548800000.0, 'train_loss': 3.695304787028377, 'epoch': 3.0})

In [51]:
#сохраненяем модель
trainer.save_model(output_dir + '/model')

Saving model checkpoint to gazeta/output/model
Configuration saved in gazeta/output/model\config.json
Model weights saved in gazeta/output/model\pytorch_model.bin


Генерация заголовков статей

In [52]:
#тестовые данные
INX = 10
input_text = dataset_test['text'][INX]
input_title = dataset_test['title'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("Text:\n" + input_text)
print("Real title: " + input_title)
print("Pred title: " + pred)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text:
Из заявления МИД РФ
: «21 августа Россия выступила с инициативой принятия Советом Безопасности ООН решения в поддержку доставки гуманитарной помощи на украинский юго-восток. Эта конструктивная российская инициатива была заблокирована США и Литвой».
В МИД РФ считают очевидной двуличие такой политики и обвиняют США и их европейских партнеров в «циничном пренебрежении судьбами мирных граждан и наплевательском отношении к международному гуманитарному праву».
Из заявления МИД РФ
: «Если США пошли против абсолютно неконфронтационного, примиряющего текста, то не остается никаких сомнений в нацеленности Вашингтона на продолжение вооруженного противостояния на Украине. Иначе как попыткой „подорвать“ гуманитарную миссию назвать это нельзя».
Real title: МИД РФ: США пытаются сорвать российскую гуманитарную миссию на Украине
Pred title: МИД РФ: США и Литва отказались от доставки гуманитарной помощи на Украине


Заголовок получился содержательнее оригинала. Моделька отработала хорошо.